In [ ]:
import cfbd

In [ ]:
import time
import cfbd
from pprint import pprint
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

from cfbd.models.division_classification import DivisionClassification
from cfbd.models.game import Game
from cfbd.models.season_type import SeasonType
from cfbd.rest import ApiException
from cfbd.models.game import Game

load_dotenv()
# Defining the host is optional and defaults to https://api.collegefootballdata.com
# See configuration.py for a list of all supported configuration parameters.
configuration = cfbd.Configuration(
    host = "https://api.collegefootballdata.com"
)

# The client must configure the authentication and authorization parameters
# in accordance with the API server security policy.
# Examples for each auth method are provided below, use the example that
# satisfies your auth use case.

# Configure Bearer authorization: apiKey
configuration = cfbd.Configuration(
    access_token = os.environ.get("CFBD_API_KEY")
)



In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = cfbd.GamesApi(api_client)
    year = 2024 
    division = cfbd.DivisionClassification('fbs')
    
    try:
        api_response = api_instance.get_games(year=year, classification=division)
        ly_api_response = api_instance.get_games(year=year-1, classification=division)
    except Exception as e:
        print("Exception when calling GamesApi->get_games: %s\n" % e)

In [ ]:
from typing import Iterable

def prepare_schedule(
    api_response: Iterable,  
    hfa: int = 3,
    decay: float = 1/3,  
) -> pd.DataFrame:
    """
    Return ['week','winner','loser','hfa_margin'] ready for add_weight().
    - Drop canceled/incomplete games (missing scores)
    - Assert neutralSite complete (per your rule after drop)
    - Winner-perspective, HFA-adjusted margin
    - Assert no ties (FBS)
    """
    
    cols = ['seasonType','week','neutralSite',
            'homeTeam','awayTeam','homePoints','awayPoints']

    # Vectorized load in one shot
    df = pd.DataFrame.from_records((g.to_dict() for g in api_response), columns=cols)

    # 1) Drop canceled / incomplete
    df = df.dropna(subset=['homePoints','awayPoints']).reset_index(drop=True)
    if df.empty:
        return pd.DataFrame(columns=['week','winner','loser','hfa_margin'])

    # 2) Fail-fast invariants
    assert not df[['seasonType','week','homeTeam','awayTeam']].isna().any().any(), \
        "Nulls in required non-score fields."
    assert not df['neutralSite'].isna().any(), \
        "neutralSite should be non-null after dropping canceled games."

    # 3) Types + postseason mapping
    df['week'] = pd.to_numeric(df['week'], errors='raise', downcast='integer')
    df.loc[df['seasonType'].eq('postseason'), 'week'] = 18
    df['week'] = df['week'].astype('int16')
    assert (df['week'] >= 1).all(), "week must be >= 1"

    # Pull arrays once 
    hp = pd.to_numeric(df['homePoints'], errors='raise').to_numpy()
    ap = pd.to_numeric(df['awayPoints'], errors='raise').to_numpy()
    ns = df['neutralSite'].astype(bool).to_numpy()
    wk = df['week'].to_numpy()
    home = df['homeTeam'].to_numpy(object)
    away = df['awayTeam'].to_numpy(object)

    # 4) Margins & outcomes
    margin = hp - ap                          # home-perspective true margin
    assert not (margin == 0).any(), "Unexpected tie in completed FBS game."
    home_field = np.where(ns, 0, hfa)         # 0 if neutral, else HFA
    adj_home = margin - home_field            # remove HFA from home side

    home_win = margin > 0
    # away_win = margin < 0  # redundant given assert

    winners = np.where(home_win, home, away)
    losers  = np.where(home_win, away, home)
    hfa_margin = np.where(home_win, adj_home, -adj_home)

    return pd.DataFrame({
        'week': wk,
        'winner': winners,
        'loser': losers,
        'hfa_margin': hfa_margin,
    })


In [ ]:
def add_weight(df: pd.DataFrame, decay: float = 1/3) -> pd.DataFrame:
    """
    Calculate weights for college football games based on team game counts and recency.
    
    Weight formula: sqrt((total_games / max_total_games) / (weeks_ago ** decay))
    Weights are normalized to sum to 100.
    
    Args:
        df: DataFrame with columns ['week', 'winner', 'loser', 'hfa_margin']
        decay: Time decay factor for recency weighting (default: 1/3)
    
    Returns:
        DataFrame with columns ['week', 'winner', 'loser', 'hfa_margin', 'weight']
        
    Performance: ~14.8x faster than naive pandas approach using:
        - pd.factorize() for efficient team encoding
        - np.bincount() for fast game counting  
        - Pure numpy operations for mathematical calculations
    """
    # Handle empty DataFrame edge case

    # --- fail-fast checks ---
    assert decay > 0, "decay must be > 0"
    assert not df[['winner','loser']].isna().any().any(), "winner/loser must be non-null"
    assert (df['week'] >= 1).all(), "week must be >= 1"
    assert len(df) > 0, "empty dataframe"
    
    if df.empty:
        return df.assign(weight=pd.Series(dtype='float64'))[
            ['week', 'winner', 'loser', 'hfa_margin', 'weight']
        ]
    
    # Extract numpy arrays once to minimize pandas overhead
    winner_vals = df['winner'].values
    loser_vals = df['loser'].values
    week_vals = df['week'].values
    
    # Efficient team encoding using pandas factorize
    both_teams = np.concatenate([winner_vals, loser_vals])
    codes, _ = pd.factorize(both_teams, sort=False)
    
    # Fast game counting using numpy bincount
    n = len(df)
    counts = np.bincount(codes)
    winner_games = counts[codes[:n]]
    loser_games = counts[codes[n:]]
    
    # Pure numpy calculations for maximum speed
    total_games = winner_games + loser_games
    weeks_ago = (week_vals.max() + 1) - week_vals
    max_games = total_games.max()
    
    # Calculate weights using vectorized operations
    if max_games > 0:
        weights = np.sqrt((total_games / max_games) / (weeks_ago ** decay))
        # Normalize to sum to 100
        weights *= (100.0 / weights.sum())
    else:
        # Edge case: no games played (shouldn't happen in real data)
        weights = np.zeros(n, dtype=np.float64)
    
    # Return result with weight column
    result = df[['week', 'winner', 'loser', 'hfa_margin']].copy()
    result['weight'] = weights
    return result

In [ ]:
df_raw = prepare_schedule(api_response)

add_weight(df_raw)

## DEV BELOW THIS CELL

In [ ]:
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import lsqr

def get_initial(schedule):

    extras = schedule[['hfa_margin', 'weight']]
    transform = schedule.drop(['hfa_margin', 'weight'], axis = 1)

    # Get a list of all unique teams
    teams = sorted(set(transform['winner'].unique()).union(transform['loser'].unique()))

    # Create a new DataFrame with teams as columns using Scipy's sparse lil_matrix
    n = len(transform)
    m = len(teams)
    x = lil_matrix((n, m), dtype=int)

    # Create a dictionary to map teams to their respective column indices
    team_indices = {team: index for index, team in enumerate(teams)}

    # Fill in the sparse matrix with 1 for winners and -1 for losers efficiently
    winners = transform['winner'].map(team_indices).values
    losers = transform['loser'].map(team_indices).values

    x[np.arange(n), winners] = 1
    x[np.arange(n), losers] = -1

    #my code
    y = extras['hfa_margin'].to_numpy()
    w = extras['weight'].to_numpy()

    xw = x.multiply(np.sqrt(w[:, np.newaxis]))
    yw = y * np.sqrt(w)

    result, istop, itn, _, _, _, _, _, _, _ = lsqr(xw, yw)

    r1_ratings = pd.DataFrame(data = {'teams': teams, 'coefs': result})
    #r1_ratings.sort_values(by=['coefs'], inplace=True, ascending=False)

    schedule.set_index('winner', inplace=True, drop = False)
    r1_ratings.set_index('teams', inplace=True, drop = False)
    with_winner = schedule.join(r1_ratings, how='left').set_index('loser', drop = False)

    with_ratings = with_winner.join(r1_ratings, how = 'left', lsuffix='_winner', rsuffix='_loser').drop(['teams_winner', 'teams_loser'], axis = 1)
    with_ratings.reset_index(inplace = True, drop = True)

    return with_ratings

In [ ]:
df_raw

In [ ]:
df_raw = add_weight(prepare_schedule(api_response))

get_initial(df_raw)

In [ ]:
def get_rating(subject, initial):
    with_ratings = initial[['winner', 'loser', 'hfa_margin', 'weight','coefs_winner', 'coefs_loser']]
    subject_mask = (with_ratings['winner'] == subject) | (with_ratings['loser'] == subject)
    subject_data = with_ratings[subject_mask].copy()
    subject_data['hfa_margin'] *= np.where(subject_data['winner'] == subject, 1, -1)
    subject_data.columns = ['team1', 'team2', 'hfa_margin', 'weight', 'rating_team1', 'rating_team2']

    subject_data['y'] = subject_data['hfa_margin']+subject_data['rating_team2']
    subject_data['x'] = 1
    x = subject_data['x'].to_numpy()
    y = subject_data['y'].to_numpy()
    w = subject_data['weight'].to_numpy()

    # Apply weights to x and y
    xw = x * np.sqrt(w)
    yw = y * np.sqrt(w)

    A = xw[:, np.newaxis]

    result, _, _, _ = np.linalg.lstsq(A, yw, rcond=0.1)

    return result[0]

def get_ratings(schedule):
    initial = get_initial(schedule)
    teams = sorted(set(schedule['winner'].unique()).union(schedule['loser'].unique()))
    output_list = list(map(lambda x: get_rating(x, initial), teams))
    ratings = pd.DataFrame(list(zip(teams, output_list)), columns=['teams', 'ratings'])
    return ratings.sort_values("ratings", axis = 0, ascending = False)

def get_error(schedule, ratings):
    error_schedule = schedule.drop(['week'], axis = 1)
    ratings.sort_values(by=['ratings'], inplace=True, ascending=False)

    error_schedule.set_index('winner', inplace=True, drop = False)
    ratings.set_index('teams', inplace=True, drop = False)
    with_winner = error_schedule.join(ratings, how='left').set_index('loser', drop = False)

    with_ratings = with_winner.join(ratings, how = 'left', lsuffix='_winner', rsuffix='_loser').drop(['teams_winner', 'teams_loser'], axis = 1)
    with_ratings.reset_index(inplace = True, drop = True)
    with_ratings['error'] = (with_ratings['hfa_margin'] - (with_ratings['ratings_winner'] - with_ratings['ratings_loser']))**2

    with_ratings.drop(['hfa_margin','ratings_winner', 'ratings_loser'], inplace = True, axis = 1)

    with_ratings2 = with_ratings.copy()

    with_ratings.columns = ['team1', 'team2', 'weight', 'error']
    with_ratings2.columns = ['team2', 'team1', 'weight', 'error']

    error_set = (pd.concat([with_ratings, with_ratings2], ignore_index=True)).drop(['team2'], axis = 1)
    ##need to factor in weight
    error_sum = pd.DataFrame(error_set.groupby(by = 'team1', axis=0).apply(lambda x: (x.weight*x.error).sum()))
    error_count = error_set.drop(['weight'], axis = 1).groupby(by = 'team1', axis=0).count()


    error_total = error_sum.join(error_count, lsuffix = "r", rsuffix = "l")
    error_total.reset_index(inplace = True)
    error_total.columns = ['team', 'error', 'games']

    error_total['rmse'] = (error_total['error']/error_total['games'])**0.5
    error_total['psudo_sd'] = ((error_total['rmse']*error_total['games'])+6*22)/(error_total['games']+22)
    error = error_total.drop(['error','games','rmse'], axis = 1)
    return error

def combined(ratings, error):
    error.set_index('team', drop = False, inplace = True)
    rating_error = ratings.join(error, how = 'left', lsuffix='_l', rsuffix='_r').drop(['teams','team'], axis = 1).reset_index()
    rating_error.columns = ['team','rating','psudo_sd']
    return rating_error

def error_hfa(x, year, week, soup=None):
    hfa = x
    if soup is None:
        schedule, _ = get_schedule(year, week=week, hfa = hfa, decay = 0)
    else:
        schedule, _ = get_schedule(year, week=week, hfa = hfa, decay = 0, soup = soup)

    ratings = get_ratings(schedule)
    return get_error(schedule, ratings)['psudo_sd'].sum()

def error_decay(x, hfa, year, week, soup=None):
    decay = x
    if soup is None:
        schedule, _ = get_schedule(year, week=week, hfa = hfa, decay = decay)
    else:
        schedule, _ = get_schedule(year, week=week, hfa = hfa, decay = decay, soup = soup)

    ratings = get_ratings(schedule)
    return get_error(schedule, ratings)['psudo_sd'].sum()


In [ ]:
prepare_schedule(api_response)

In [ ]:
schedule = add_weight(prepare_schedule(api_response))
ratings = get_ratings(schedule)
error = get_error(schedule, ratings)

combined(ratings, error)

### HANK'S CODE BELOW

In [ ]:
def n_max(given_list, k, default = 0):
    length = len(given_list)+1
    if k >= length:
        return default
    given_list.sort()
    return given_list[-k]

def n_min(given_list, k, default = 0):
    length = len(given_list)+1
    if k >= length:
        return default
    given_list.sort()
    return given_list[-(length - k)]

class Team:
    def __init__(self, name, victory, defeat):
        self.name = name
        self.victory = victory
        self.defeat = defeat

    def get_wins(self):
        return len(self.victory)

    def get_victories(self):
        return self.victory

    def get_losses(self):
        return len(self.defeat)

    def get_defeats(self):
        return self.defeat

    def __repr__(self):
        return f"Team: {self.name}, Victory: {self.victory}, Defeat: {self.defeat}"


def get_worster(api_response, ly_api_response):
    schedule = add_weight(prepare_schedule(api_response))
    team_dict = {}
    team_list = set(schedule['winner']).union(schedule['loser'])

    for i in team_list:
        victory = schedule.loc[(schedule['winner']==i)]['loser'].tolist()
        defeat = schedule.loc[(schedule['loser']==i)]['winner'].tolist()
        team_dict[i] = Team(i, victory, defeat)

    team_df = (pd.DataFrame(team_list, columns = ['team'])
     .assign(wins =
             list(map(lambda x: team_dict[x].get_wins(), team_list)))
     .assign(losses =
             list(map(lambda x: team_dict[x].get_losses(), team_list)))
     .assign(wins_from_1best =
             list(map(lambda y: max(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_victories())),default=0)
                      , team_list
                     )))
     .assign(wins_from_1worst =
             list(map(lambda y: min(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_defeats())),default=0)
                      , team_list
                     )))
     .assign(wins_from_2best =
             list(map(lambda y: n_max(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_victories())),2,default=0)
                      , team_list
                     )))
     .assign(wins_from_2worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_defeats())),2,default=0)
                      , team_list
                     )))
     .assign(wins_from_3best =
             list(map(lambda y: n_max(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_victories())),3,default=0)
                      , team_list
                     )))
     .assign(wins_from_3worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_defeats())),3,default=0)
                      , team_list
                     )))
     .assign(wins_from_4best =
             list(map(lambda y: n_max(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_victories())),4,default=0)
                      , team_list
                     )))
     .assign(wins_from_4worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_defeats())),4,default=0)
                      , team_list
                     )))
     .assign(wins_from_5best =
             list(map(lambda y: n_max(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_victories())),5,default=0)
                      , team_list
                     )))
     .assign(wins_from_5worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_defeats())),5,default=0)
                      , team_list
                     )))
     .assign(wins_from_6best =
             list(map(lambda y: n_max(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_victories())),6,default=0)
                      , team_list
                     )))
     .assign(wins_from_6worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_defeats())),6,default=0)
                      , team_list
                     )))
     .assign(wins_from_7best =
             list(map(lambda y: n_max(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_victories())),7,default=0)
                      , team_list
                     )))
     .assign(wins_from_7worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_defeats())),7,default=0)
                      , team_list
                     )))
     .assign(wins_from_8best =
             list(map(lambda y: n_max(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_victories())),8,default=0)
                      , team_list
                     )))
     .assign(wins_from_8worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_defeats())),8,default=0)
                      , team_list
                     )))
     .assign(wins_from_9best =
             list(map(lambda y: n_max(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_victories())),9,default=0)
                      , team_list
                     )))
     .assign(wins_from_9worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_defeats())),9,default=0)
                      , team_list
                     )))
     .assign(wins_from_10best =
             list(map(lambda y: n_max(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_victories())),10,default=0)
                      , team_list
                     )))
     .assign(wins_from_10worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_defeats())),10,default=0)
                      , team_list
                     )))
     .assign(wins_from_11best =
             list(map(lambda y: n_max(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_victories())),11,default=0)
                      , team_list
                     )))
     .assign(wins_from_11worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_defeats())),11,default=0)
                      , team_list
                     )))
     .assign(wins_from_12best =
             list(map(lambda y: n_max(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_victories())),12,default=0)
                      , team_list
                     )))
     .assign(wins_from_12worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: team_dict[x].get_wins(), team_dict[y].get_defeats())),12,default=0)
                      , team_list
                     )))
    )

    ly_schedule = add_weight(prepare_schedule(ly_api_response))
    ly_team_dict = {}
    ly_team_list = set(ly_schedule['winner']).union(ly_schedule['loser'])

    for i in ly_team_list:
        victory = ly_schedule.loc[(ly_schedule['winner']==i)]['loser'].tolist()
        defeat = ly_schedule.loc[(ly_schedule['loser']==i)]['winner'].tolist()
        ly_team_dict[i] = Team(i, victory, defeat)


    ly_team_df = (pd.DataFrame(ly_team_list, columns = ['team'])
     .assign(ly_wins =
             list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_list)))
     .assign(ly_losses =
             list(map(lambda x: ly_team_dict[x].get_losses(), ly_team_list)))
     .assign(ly_wins_from_1best =
             list(map(lambda y: max(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_victories())),default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_1worst =
             list(map(lambda y: min(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_defeats())),default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_2best =
             list(map(lambda y: n_max(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_victories())),2,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_2worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_defeats())),2,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_3best =
             list(map(lambda y: n_max(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_victories())),3,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_3worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_defeats())),3,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_4best =
             list(map(lambda y: n_max(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_victories())),4,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_4worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_defeats())),4,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_5best =
             list(map(lambda y: n_max(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_victories())),5,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_5worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_defeats())),5,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_6best =
             list(map(lambda y: n_max(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_victories())),6,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_6worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_defeats())),6,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_7best =
             list(map(lambda y: n_max(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_victories())),7,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_7worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_defeats())),7,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_8best =
             list(map(lambda y: n_max(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_victories())),8,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_8worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_defeats())),8,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_9best =
             list(map(lambda y: n_max(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_victories())),9,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_9worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_defeats())),9,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_10best =
             list(map(lambda y: n_max(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_victories())),10,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_10worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_defeats())),10,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_11best =
             list(map(lambda y: n_max(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_victories())),11,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_11worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_defeats())),11,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_12best =
             list(map(lambda y: n_max(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_victories())),12,default=0)
                      , ly_team_list
                     )))
     .assign(ly_wins_from_12worst =
             list(map(lambda y: n_min(
                 list(map(lambda x: ly_team_dict[x].get_wins(), ly_team_dict[y].get_defeats())),12,default=0)
                      , ly_team_list
                     )))
    )

    team_df.set_index('team',drop=True,inplace=True)
    ly_team_df.set_index('team',drop=True,inplace=True)

    joined = team_df.join(ly_team_df#, validate = 'one_to_one'
                         )

    joined = (
        joined.sort_values(by = (['wins',
                                'wins_from_1best', 'wins_from_1worst',
                                'wins_from_2best', 'wins_from_2worst',
                                'wins_from_3best', 'wins_from_3worst',
                                'wins_from_4best', 'wins_from_4worst',
                                'wins_from_5best', 'wins_from_5worst',
                                'wins_from_6best', 'wins_from_6worst',
                                'wins_from_7best', 'wins_from_7worst',
                                'wins_from_8best', 'wins_from_8worst',
                                'wins_from_9best', 'wins_from_9worst',
                                'wins_from_10best', 'wins_from_10worst',
                                'wins_from_11best', 'wins_from_11worst',
                                'wins_from_12best', 'wins_from_12worst',
                                'ly_wins',
                                'ly_wins_from_1best', 'ly_wins_from_1worst',
                                'ly_wins_from_2best', 'ly_wins_from_2worst',
                                'ly_wins_from_3best', 'ly_wins_from_3worst',
                                'ly_wins_from_4best', 'ly_wins_from_4worst',
                                'ly_wins_from_5best', 'ly_wins_from_5worst',
                                'ly_wins_from_6best', 'ly_wins_from_6worst',
                                'ly_wins_from_7best', 'ly_wins_from_7worst',
                                'ly_wins_from_8best', 'ly_wins_from_8worst',
                                'ly_wins_from_9best', 'ly_wins_from_9worst',
                                'ly_wins_from_10best', 'ly_wins_from_10worst',
                                'ly_wins_from_11best', 'ly_wins_from_11worst',
                                'ly_wins_from_12best', 'ly_wins_from_12worst']),
                         axis=0, ascending=([False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False,
                                             False, False]) )
    )

    joined.reset_index(inplace = True, drop = False)

    return joined

In [ ]:
get_worster(api_response, ly_api_response)